# Generating images using Amazon Titan Image Generator v2

> ☝️ This notebook should work well with the **`Data Science 3.0`** kernel in Amazon SageMaker Studio and with the **`conda_python3`** in a Amazon SageMaker Notebook Instance.

---

# Introduction

Welcome to the Amazon Titan Image Generator v2 Workshop! In this hands-on session, we'll explore the powerful capabilities of Amazon Titan Generator v2 to create compelling marketing materials for Octank, a premium dog food company.

### Workshop Objectives

By the end of this workshop, you will:

1. Understand the key features of Amazon Titan Image Generator v2

2. Learn how to use these features to create  marketing materials for a real-world scenario

3. Gain hands-on experience with the Amazon Bedrock API for image generation tasks

### Features We'll Use

During the workshop, we'll leverage the following features of Amazon Titan Image Generator v2:

1. **Text-to-Image**: To create a symbolic dog image for Octank

2. **Image Conditioning**: To create initial product package designs based on previous product's marketing images

3. **Color Conditioning**: To generate product design images adhere to Octank's brand color palette

4. **Background Removal**: To isolate product images for use in various marketing materials.



## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, make sure you've completed the setup in the ['Getting Started' section of the root README](https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/README.md#Getting-started). ⚠️ ⚠️ ⚠️


In [ ]:
# Built-in libraries
import base64
import io
import json
import os
import sys

# External dependencies
import boto3
import botocore
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

boto3_bedrock = boto3.client('bedrock-runtime')

In [ ]:
# Define plot function
def plot_images(base_images, prompt=None, seed=None, ref_image_path=None, color_codes=None, original_title=None, processed_title=None):
    if ref_image_path and color_codes:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        num_subplots = 3
    elif ref_image_path or color_codes:
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        num_subplots = 2
    else:
        fig, axes = plt.subplots(1, 1, figsize=(6, 5))
        num_subplots = 1
    
    axes = np.array(axes).ravel() 
    
    current_subplot = 0
    
    if color_codes:
        num_colors = len(color_codes)
        color_width = 0.8 / num_colors
        for i, color_code in enumerate(color_codes):
            x = i * color_width
            rect = plt.Rectangle((x, 0), color_width, 1, facecolor=f'{color_code}', edgecolor='white')
            axes[current_subplot].add_patch(rect)
        axes[current_subplot].set_xlim(0, 0.8)
        axes[current_subplot].set_ylim(0, 1)
        axes[current_subplot].set_title('Color Codes')
        axes[current_subplot].axis('off')
        current_subplot += 1
    
    if ref_image_path:
        reference_image = Image.open(ref_image_path)
        max_size = (512, 512)
        reference_image.thumbnail(max_size)
        axes[current_subplot].imshow(np.array(reference_image))
        axes[current_subplot].set_title(original_title or 'Reference Image')
        axes[current_subplot].axis('off')
        current_subplot += 1
    
    axes[current_subplot].imshow(np.array(base_images[0]))
    if processed_title:
        axes[current_subplot].set_title(processed_title)
    elif ref_image_path and seed is not None:
        axes[current_subplot].set_title(f'Image Generated Based on Reference\nSeed: {seed}')
    elif seed is not None:
        axes[current_subplot].set_title(f'Image Generated\nSeed: {seed}')
    else:
        axes[current_subplot].set_title('Processed Image')
    axes[current_subplot].axis('off')
    
    if prompt:
        print(f"Prompt: {prompt}\n")
    
    plt.tight_layout()
    plt.show()

# Use Case

## Text-to-Image

First, let's use TIG v2 Text-to-Image capability to create a symbolic dog image for Octank. 

Like TIG v1, in text-to-image mode, we provide a text description (prompt) of the image that should be generated. The parameters can be specified include:

* `cfgscale` - determines how much the final image reflects the prompt
* `seed` - a number used to initialize the generation, using the same seed with the same prompt + settings combination will produce the same results
* `numberOfImages` - the number of times the image is sampled and produced
* `quality` - determines the output image quality (`standard` or `premium`)

> ☝️ For more information on available input parameters for the model, refer to the [Amazon Bedrock User Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html#model-parameters-titan-img-request-body) (Inference parameters > Amazon Titan image models > Model invocation request body fields).

The cell below invokes the Amazon Titan Image Generator v2 model through Amazon Bedrock to create a symbolic dog image for Octank:

In [ ]:
prompt = "Generate a photo of a happy and healthy American Eskimo sitting on the grass with a food bowl full of kibble nearby"
negative_prompts = "low resolution, low quality"

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,                    # Required
            "negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 1024,        # Supported height list in the docs 
            "width": 1024,         # Supported width list in the docs
            "cfgScale": 8,       # Range: 1.0 (exclusive) to 10.0
            "seed": 42     # Range: 0 to 214783647
        }
    }
)

# Make model request
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

# Process the image
response_body = json.loads(response.get("body").read())


# plot images
images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

plot_images(images, processed_title="Generated Image")

## Image Conditioning

In our previous section, we generated a realistic photo of a happy Eskimo dog enjoying Octank's dog food. Now, let's explore how we can use image conditioning to create variations and expand our visual brand. 

Amazon Titan Image Generator v2 offers two image conditioning modes:

- Canny Edge: Extract prominent edges from the reference image to guide the generation process.

- Segmentation: Define specific regions/objects within the reference image for the model to generate content aligned with those areas.

Main parameters can be specified include:

* `text` - (Required) A text prompt to generate the image, must be <= 512 characters
* `negativeText` - (Optional) A text prompt to define what not to include in the image, must be <= 512 characters
* `conditionImage` - (Optional, V2 only) A base64-encoded string representing an input image to guide the layout and composition of the generated image
* `controlMode` - (Optional, V2 only) Specifies the type of conditioning mode: `CANNY_EDGE` (default) or `SEGMENTATION`
* `controlStrength` - (Optional, V2 only) Determines how similar the generated image should be to the condition image, range 0 to 1.0 (default: 0.7)
* `numberOfImages` - The number of images to generate
* `height` - The height of the generated image(s)
* `width` - The width of the generated image(s)
* `cfgScale` - Determines how closely the image adheres to the prompt
* `seed` - An integer used to initialize the image generation process

Note: The longer side of the image resolution must be <= 1,408 pixels. If `controlMode` or `controlStrength` are provided, `conditionImage` must also be provided.

#### 1. Canny Edge Mode:

We will use our Eskimo dog image to generate a cartoon-style dog in a fairy world, keeping the same general pose and composition.

In [ ]:
# Define the prompt, reference image
prompt = "a cartoon dog with dog food in a fairy world"
reference_image_path = "images/generated_dog.png"
seed = 0 # Can be any random number between 0 to 214783647

In [ ]:
# Encode the reference image
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
    
# Generate image condition on reference image
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,  # Required
            "conditionImage": reference_image_base64, # Optional
            "controlMode": "CANNY_EDGE", # Optional: CANNY_EDGE | SEGMENTATION
            "controlStrength": 0.7,  # Range: 0.2 to 1.0,
        },
        "imageGenerationConfig": {
                "numberOfImages": 1,
                "seed": seed,
            }
        
    }
)

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

# plot output
plot_images(response_images, ref_image_path = reference_image_path)

#### 2. Segmentation Mode:

We'll start with an image of a dog, house, human and dog food bowl. Then we'll generate varations that keep these components consistent but alter other aspects of the scene.

In [ ]:
# Define the prompt, reference image 
prompt = "a happy golden retriever dog loves his dog food during sunset hours"
reference_image_path = "images/dog_food.png"
seed = 42 # Can be any random number between 0 to 214783647

In [ ]:
# Encode the reference image
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
    
# Generate image condition on reference image
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,  # Required
            "conditionImage": reference_image_base64, # Optional
            "controlMode": "SEGMENTATION", # Optional: CANNY_EDGE | SEGMENTATION
            "controlStrength": 0.7,  # Range: 0.2 to 1.0,
        },
        "imageGenerationConfig": {
                "numberOfImages": 1,
                "seed": seed,
            }
        
    }
)

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

# plot output
plot_images(response_images, ref_image_path = reference_image_path)

## Color Conditioning

Octank has estabished a strong brand identity with a specific color palette. Their primary colors are represented by the following hex codes: #ffe599, #3d85c6, #eeeeee. They want to ensure all their marketing images adhere to this color scheme to strengthen brand recognition. 

Amazon Titan Image Generator v2's color conditioning feature allows users to generate images that follow a specified color palette. This can be done with or without a reference image.

Here's a summary of the parameters for color conditioning:

* `text` - (Required) A text prompt to generate the image, must be <= 512 characters
* `colors` - (Required) A list of 1 to 10 hex color codes to specify colors in the generated image
* `negativeText` - (Optional) A text prompt to define what not to include in the image, must be <= 512 characters
* `referenceImage` - (Optional) A base64-encoded string representing an input image to guide the color palette of the generated image
* `numberOfImages` - The number of images to generate
* `height` - The height of the generated image(s)
* `width` - The width of the generated image(s)
* `cfgScale` - Determines how closely the image adheres to the prompt
* `seed` - An integer used to initialize the image generation process


In [ ]:
# Define the prompt, color code 
prompt = "A happy American Eskimo runs in the blossom yard"
hex_color_code = ['#ffe599', '#3d85c6', '#eeeeee']
seed = 50 # Can be any random number between 0 to 214783647

In [ ]:
# Generate image condition on color palette
body = json.dumps({
    "taskType": "COLOR_GUIDED_GENERATION",
    "colorGuidedGenerationParams": {
        "text": prompt,
        "colors": hex_color_code
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "seed": seed,
    }
})

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

# plot output
plot_images(response_images, color_codes = hex_color_code)

Additionally, users can upload a single reference image that is similar to their desired output. The model will then generate images that follow the style and fashion of this reference image while incorporating the specified color palette.

In [ ]:
# Define the prompt, reference image, color code and path to store the generated images
reference_image_path = "images/impression_style.png"
prompt = "A happy American Eskimo runs in the blossom yard"
hex_color_code = ['#ffe599', '#3d85c6', '#eeeeee']
seed = 121 # Can be any random number between 0 to 214783647

In [ ]:
# Encode the reference image
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
    
    
# Generate image condition on color palette
body = json.dumps({
    "taskType": "COLOR_GUIDED_GENERATION",
    "colorGuidedGenerationParams": {
        "text": prompt,
        "colors": hex_color_code,
        "referenceImage": reference_image_base64
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "seed": seed,
    }
})

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]


# plot output
plot_images(response_images, ref_image_path = reference_image_path, color_codes = hex_color_code)

## Background Removal

Octank has professional photos of their new gourmet dog food. They want to use these images across various marketing materials with different background. In our last use case, we will use Background Removal feature from Titan Image Generator v2 to help Ocktank isolate its product image from their original backgrond.

To use this feature, you just need to provide the image the model needs to work with. 

In [ ]:
# Define image needs to be processed and path to store the generated images
reference_image_path = "images/dog_food_pack.png"

In [ ]:
# Read image from file and encode it as base64 string.
with open(reference_image_path, "rb") as image_file:
    input_image = base64.b64encode(image_file.read()).decode('utf8')

body = json.dumps({
    "taskType": "BACKGROUND_REMOVAL",
    "backgroundRemovalParams": {
        "image": input_image,
    }
})

response = boto3_bedrock.invoke_model(
    body=body, 
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

response_body = json.loads(response.get("body").read())
response_images = [
    Image.open(io.BytesIO(base64.b64decode(base64_image)))
    for base64_image in response_body.get("images")
]

# plot output
plot_images(response_images, ref_image_path= reference_image_path, original_title='Original Image', processed_title='Processed Image without Background')

# Summary

In this workshop, we explored the powerful features of Amazon Titan Image Generator v2 through the lens of Octank, a premium dog food company. We covered:

- Text-to-Image Generaion
- Image Conditioning
- Color Conditioning
- Background Removal

These tools enable Octank to efficiently create diverse, high-quality visuals for their marketing campaigns, maintaining brand consistency while adapting to various styles.

You can now leverage this GenAI-powered image generation to enhance your own creative workflows!